In [1]:
import pytorch_lightning as pl
import torch.nn as nn
from transformers import ElectraModel, AutoTokenizer
import torch

LABELS = ['불평/불만',
 '환영/호의',
 '감동/감탄',
 '지긋지긋',
 '고마움',
 '슬픔',
 '화남/분노',
 '존경',
 '기대감',
 '우쭐댐/무시함',
 '안타까움/실망',
 '비장함',
 '의심/불신',
 '뿌듯함',
 '편안/쾌적',
 '신기함/관심',
 '아껴주는',
 '부끄러움',
 '공포/무서움',
 '절망',
 '한심함',
 '역겨움/징그러움',
 '짜증',
 '어이없음',
 '없음',
 '패배/자기혐오',
 '귀찮음',
 '힘듦/지침',
 '즐거움/신남',
 '깨달음',
 '죄책감',
 '증오/혐오',
 '흐뭇함(귀여움/예쁨)',
 '당황/난처',
 '경악',
 '부담/안_내킴',
 '서러움',
 '재미없음',
 '불쌍함/연민',
 '놀람',
 '행복',
 '불안/걱정',
 '기쁨',
 '안심/신뢰']

device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
class KOTEtagger(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.electra = ElectraModel.from_pretrained("beomi/KcELECTRA-base").to(device)
        self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")
        self.classifier = nn.Linear(self.electra.config.hidden_size, 44).to(device)
        
    def forward(self, text:str):
        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=512,
          return_token_type_ids=False,
          padding="max_length",
          return_attention_mask=True,
          return_tensors='pt',
        ).to(device)
        output = self.electra(encoding["input_ids"], attention_mask=encoding["attention_mask"])
        output = output.last_hidden_state[:,0,:]
        output = self.classifier(output)
        output = torch.sigmoid(output)
        torch.cuda.empty_cache()
        
        return output

In [5]:
trained_model = KOTEtagger()

Some weights of the model checkpoint at beomi/KcELECTRA-base were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model_path = './model/kote_pytorch_lightning.bin'
trained_model.load_state_dict(torch.load(model_path)) # <All keys matched successfully>라는 결과가 나오는지 확인!

preds = trained_model(
"""재미있어요! 재미는 확실히 있는데 뭐랄까... 너무 정신 없달까...ㅋㅋ"""
)[0]

for l, p in zip(LABELS, preds):
    if p>0.4:
        print(f"{l}: {p}")

불평/불만: 0.650226891040802
안타까움/실망: 0.7632462382316589
어이없음: 0.4877132177352905
즐거움/신남: 0.858988881111145
당황/난처: 0.605267345905304
재미없음: 0.48861661553382874
기쁨: 0.4897276759147644


In [ ]:
# 푸른 머리칼의 살인마 - https://select.ridibooks.com/book/4711000002 
# 무기모토 산포는 오늘이 좋아 - https://select.ridibooks.com/book/2066002965
# 피리술사 - https://select.ridibooks.com/book/3421001211
# 티끌 같은 나 - https://select.ridibooks.com/book/3988000011
# 체리 - https://select.ridibooks.com/book/3988000012
# 가장 무서운 이야기 게임 - https://select.ridibooks.com/book/4765000001
# 불멸의 파우스트 - https://select.ridibooks.com/book/1242000920
# 기억 1 - https://select.ridibooks.com/book/1242000932
# 달콤한 복수 주식회사 - https://select.ridibooks.com/book/1242001052
# 핵을 들고 도망친 101세 노인 - https://select.ridibooks.com/book/1242000882

In [28]:
book_lst = [
"성을 다스리는 젊고 부유한 영주가 어느 날 도끼로 살해당하는 참혹한 사건이 벌어진다. \
영주가 끔찍하게 죽었다는 소식에 성의 안팎은 혼란에 빠진다. 마침 비슷한 시기에 마을의 젊은 여자들이 하나둘씩 살해당했는데, \
그 사건들이 서로 관련 있을 거라는 소문이 나돈다. 가까운 거리에서 살인마를 본 유일한 목격자는 증언한다. 청록색 고급 드레스를 입은 채 도끼를 \
능숙하게 쓰는 어떤 여자였다고. 그러고서 목격자는 사건에 얽힌 충격적인 얘기를 풀어놓는데……. 탄탄한 소재 구성력과 완성도 높은 서사로 독자를 \
사로잡는 조예은 작가가 펼쳐 보이는 타임루프 미스터리 판타지의 절정. 평소 압도적 스토리를 찾느라 갈증을 느낀다면 지금 바로 이 소설을 마주해야 한다.",
"친구의 안 좋은 소식을 듣고 조용히 눈물짓거나, 일터에서 좋아하는 과자를 발견한 것으로 하루 종일 행복해하거나, 이별의 아픔으로 울다가도 애인 없이 \
보내는 주말의 상쾌함을 느끼는 산포의 매일. 주변에서는 ‘멍하다’, ‘많이 먹는다’, ‘얼빠졌다’ 등의 잔소리를 듣지만 이런 자신의 모습까지 사랑하며 있는 \
그대로 받아들이는 산포는 ‘산포’ 자신이기에 행복하다. 주말에는 꼭 낮잠을 자거나, 짠 음식을 먹었을 때는 반드시 달달한 디저트를 먹거나, 어떨 땐 과감하게 \
꾀병을 부리며 출근을 빼먹었음에도 죄책감에 몸부림치는 인간적인 면모까지. 남의 의견에 휘둘리지 않는 대신 스스로에게 취하는 산포의 독특한 행보는 읽으면 \
읽을수록 독자를 산포의 사랑스러운 매력 속으로 이끈다.",
"""에도의 미시마야에서 한 아가씨가 기이한 이야기를 모으고 있다. 그곳에 한 사람씩 자신이 겪은, 누구에게도 털어놓지 못했던 이야기를 하기 위해 사람들이 찾아온다. 가슴속에 맺혀 있던 이야기를 털어놓은 사람들은 마치 보이지 않는 짐을 부려놓은 듯 모종의 평온을 얻는 것 같다. 그 평온의 온기가 이야기를 듣는 이의 마음에도 등불을 밝혀 준다.
영혼이 부서질 정도로 비극적인 일을 겪은 이에게 어지간한 위로나 격려는 별 소용이 없으며, 그보다는 이런 식으로 이야기들에서 실을 자아내 스스로 자신의 영혼을 꿰매어 수선할 수 있도록 도와주는 것이 좋지 않을까, 를 고찰해 보고자 이 시리즈를 쓰기 시작했다고 작가는 밝히고 있다.""",
"""《티끌 같은 나》는 등장인물의 고난과 이에 따른 갈등 혹은 단순히 사랑을 다룬 소설로서 저자 특유의 위트와 유머를 느끼며 가볍게 읽어도 좋다. 한편 그 속에 숨어 있는 야망을 이루고자 하는 등장인물의 거짓과 배신, 외로움, 좌절, 진실과 사랑, 기대, 희망이 끊임없이 갈등을 일으키며 변화하는 모습을 냉정하고 객관적으로 들여다볼 수도 있을 것이다. 이 책을 읽는 동안 울고 웃으며, 과거와 현재 그리고 앞으로 다가올 미래 사회와 그 속에서 살아갈 우리의 진짜 모습을 발견해 내기를 바란다.""",
"""에밀리를 만난 건 2003년, 클리블랜드의 대학에 들어갔을 때다. 좀처럼 학교에 적응하지 못하고 방황하다 그녀를 본 순간 단번에 이끌렸고, 그렇게 우리는 서로를 아프게 할 운명으로 엮인다. 나는 마약에 취해 에밀리와 사랑을 나누며 현실에서 도피하다 의료 특기병으로 군대에 입대한다. 하지만 나와 에밀리 그 누구에게도 좋지 않은 결정이었다. 에밀리와 결혼하고 이라크에 파병되어 갔지만 의료 특기병으로서 준비되지 않았고, 하나씩 둘씩 죽어 가는 동료들을 바라보는 일 말고는 아무것도 할 수 없었다. 나는 영웅이 아니었다. 아무것도 아니었다. 군복무를 마치고 집으로 돌아와서도 에밀리와 함께 헤로인에 중독된 채 서서히 삶의 낭떠러지로 떨어지는 나날이 이어지는데…….""",
"""희미한 촛불 아래, 남자 둘과 여자 하나, 세 사람이 앉아 있다. 얼핏 봐도 접점이 하나도 없어 보이는 세 사람. 이들은 연신 입술을 핥고 무언가 초조한 듯 불안한 표정을 짓고 있다. 곧이어 어디선가, 목소리가 들려온다. “자, 지금부터 시작하겠습니다.”""",
"""『파우스트』는 20대의 괴테가 구상하고, 80대의 괴테가 완성한 괴테 문학의 정점에 놓인 작품이다. 일개의 문학 작품으로서뿐 아니라, 괴테 자신의 역사관과 문명관, 신화관, 종교관 전반을 아우른다는 점에서 괴테 연구의 핵심 저술로 알려져 있다. 실제로 괴테가 써온 수많은 시와 소설, 희곡 등의 문학 작품과, 여행기, 편지 등의 개인적인 기록이 『파우스트』와 밀접하게 연결되어 다양하게 해석된다. 『파우스트』를 그 자체로 괴테의 지식과 사상의 총체라고 부를 수 있는 이유이다. 이 책은 『파우스트』의 작중 인물과, 전개 등 기본적인 이해를 돕는 1부와, 신비주의, 종교적 개념, 아름다운 여성상 등 작품에 담긴 심도 깊은 사상적 배경을 분석한 2부로 구성되었다(총 9장). 특히 2부 마지막 장에서는 『파우스트』의 현대적인 의미까지 다루고 있다.""",
"""주인공 르네 톨레다노는 고등학교에서 역사를 가르치는 교사이다. 그는 센강 유람선 공연장 <판도라의 상자>에 갔다가 퇴행 최면의 대상자로 선택당한다. 최면에 성공해 무의식의 복도에 늘어선 기억의 문을 열 수 있게 된 르네. 문 너머에서 엿본 기억은, 제1차 세계 대전의 전장에서 목숨을 잃은 그의 전생이었다. 최면이 끝난 후에도 너무나 생생하고 강렬한 기억에 시달리던 그는 몸싸움에 휘말려 의도치 않게 사람을 죽이고 경찰에 자수할지 말지 고민하며 초조한 나날을 보내게 된다.
한편 르네는 자신에게 총 111번의 전생이 있었다는 사실을 알고 제1차 세계 대전 참전병 외에도 여러 기억의 문을 열어 본다. 그중에서도 최초의 전생은 놀랍게도 현대인이 <아틀란티스>라고 부르는 전설 속의 섬에 사는 남자 게브였다. 아틀란티스가 바닷속에 잠겨 버렸다고 알고 있는 르네는 어떻게든 게브를 구하고 싶어 하고, <판도라의 상자> 무대에서 만났던 최면사 오팔이 르네의 조력자를 자처한다. 현생에서는 경찰에 쫓기며 정신병자 취급을 받고, 전생에서는 대홍수가 예고된 가운데 과연 르네와 게브의 운명은?""",
"""스웨덴 스톡홀름에 사는 빅토르는 교활하고 위선적인 미술품 거래인으로, 비열한 방법으로 아내의 재산을 빼앗고 이혼한다. 또 창녀와의 관계에서 낳은 아들 케빈을 죽이려고 케냐 사바나에 데리고 가서 버린다. 케빈은 원주민 치유사 올레 음바티안의 구조를 받아 마사이 전사로 거듭난다. 하지만 성인식에 할례가 포함되어 있다는 말에 기겁하여 다시 스웨덴으로 돌아온다. 우연히 빅토르의 전 아내 옌뉘를 만나게 된 케빈. 의기투합한 두 사람은 복수를 꿈꾸는데, 이들 앞에 나타난 것은 복수를 대행하는 <달콤한 복수 주식회사>의 CEO 후고다. 후고는 양아들을 찾아 케냐에서 스웨덴으로 건너온 올레 음바티안과 함께 두 사람을 위한 복수를 계획한다.""",
"""스웨덴의 시골 마을에서 태어났지만 백 년이 넘는 세월을 살면서 본의 아니게 세계사의 주요 사건에 끼어들게 된 주인공 알란 칼손. 백 살 생일날 양로원 창문을 넘어 도망쳤던 알란이 이번에는 백 한 살 생일날 열기구를 탔다가 조난당하며 새로운 모험을 시작한다. 『핵을 들고 도망친 101세 노인』은 요나손의 통산 네 번째 소설이다."""    
]

In [62]:
def get_score_label(preds):
    sort_idx = torch.argsort(preds, descending=True).numpy()
    scores = preds[sort_idx]
    labels = np.array(LABELS)[sort_idx.astype(int)]
    return scores, labels
    

In [63]:
for index, text in enumerate(book_lst):
    print(f'{index+1}th book')
    print(f'text : \n{text}')
    
    preds = trained_model(text)[0]
    scores, labels = get_score_label(preds)
    
    
    for label, score in zip(labels, scores):
        print(f'label : {label} // score : {score}')
    print("================")

1th book
text : 
성을 다스리는 젊고 부유한 영주가 어느 날 도끼로 살해당하는 참혹한 사건이 벌어진다. 영주가 끔찍하게 죽었다는 소식에 성의 안팎은 혼란에 빠진다. 마침 비슷한 시기에 마을의 젊은 여자들이 하나둘씩 살해당했는데, 그 사건들이 서로 관련 있을 거라는 소문이 나돈다. 가까운 거리에서 살인마를 본 유일한 목격자는 증언한다. 청록색 고급 드레스를 입은 채 도끼를 능숙하게 쓰는 어떤 여자였다고. 그러고서 목격자는 사건에 얽힌 충격적인 얘기를 풀어놓는데……. 탄탄한 소재 구성력과 완성도 높은 서사로 독자를 사로잡는 조예은 작가가 펼쳐 보이는 타임루프 미스터리 판타지의 절정. 평소 압도적 스토리를 찾느라 갈증을 느낀다면 지금 바로 이 소설을 마주해야 한다.
label : 놀람 // score : 0.8257126212120056
label : 신기함/관심 // score : 0.7138635516166687
label : 감동/감탄 // score : 0.6712079644203186
label : 기대감 // score : 0.659710705280304
label : 공포/무서움 // score : 0.6580700278282166
label : 비장함 // score : 0.5576246380805969
label : 불안/걱정 // score : 0.5548111796379089
label : 경악 // score : 0.548956573009491
label : 깨달음 // score : 0.5375243425369263
label : 존경 // score : 0.4202558696269989
label : 없음 // score : 0.30505236983299255
label : 의심/불신 // score : 0.2776689827442169
label : 불쌍함/연민 // score : 0.2733215093612671
label : 안타까움/실망 // score : 0.27052992582321167
label : 증오/혐오 //

label : 슬픔 // score : 0.9483927488327026
label : 절망 // score : 0.9107198715209961
label : 힘듦/지침 // score : 0.8680214881896973
label : 안타까움/실망 // score : 0.831851601600647
label : 서러움 // score : 0.7991821765899658
label : 패배/자기혐오 // score : 0.7768756747245789
label : 불쌍함/연민 // score : 0.7536731362342834
label : 불안/걱정 // score : 0.7234963178634644
label : 깨달음 // score : 0.5984297394752502
label : 비장함 // score : 0.5748739838600159
label : 죄책감 // score : 0.43226829171180725
label : 화남/분노 // score : 0.40619587898254395
label : 한심함 // score : 0.3235631287097931
label : 짜증 // score : 0.31822144985198975
label : 당황/난처 // score : 0.31082215905189514
label : 불평/불만 // score : 0.2912217378616333
label : 지긋지긋 // score : 0.29115504026412964
label : 의심/불신 // score : 0.2643534541130066
label : 부끄러움 // score : 0.25889068841934204
label : 공포/무서움 // score : 0.24882154166698456
label : 없음 // score : 0.23090142011642456
label : 증오/혐오 // score : 0.20672233402729034
label : 어이없음 // score : 0.2060417979955673

label : 없음 // score : 0.8240042328834534
label : 증오/혐오 // score : 0.5307977795600891
label : 경악 // score : 0.5241754055023193
label : 공포/무서움 // score : 0.4389022886753082
label : 놀람 // score : 0.435573548078537
label : 불안/걱정 // score : 0.43009451031684875
label : 불쌍함/연민 // score : 0.3891091048717499
label : 비장함 // score : 0.388617604970932
label : 화남/분노 // score : 0.37170782685279846
label : 역겨움/징그러움 // score : 0.3518761694431305
label : 깨달음 // score : 0.34792494773864746
label : 당황/난처 // score : 0.30089637637138367
label : 신기함/관심 // score : 0.29488155245780945
label : 안타까움/실망 // score : 0.272599458694458
label : 어이없음 // score : 0.224955216050148
label : 절망 // score : 0.20922358334064484
label : 슬픔 // score : 0.20450107753276825
label : 의심/불신 // score : 0.19069653749465942
label : 한심함 // score : 0.16580095887184143
label : 우쭐댐/무시함 // score : 0.15558059513568878
label : 감동/감탄 // score : 0.14940893650054932
label : 짜증 // score : 0.1382446140050888
label : 아껴주는 // score : 0.12017872929573